In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#Race Results

In [4]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_race`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [5]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP=GP[GP["Position"]!=0]
GP.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,30,L LAWSON,LAW,lawson,RB,6692FF,rb,Liam,Lawson,Liam Lawson,None,NZL,9,9,19,NaT,NaT,NaT,0 days 00:01:10.563000,Finished,2,2024,United States Grand Prix
1,22,Y TSUNODA,TSU,tsunoda,RB,6692FF,rb,Yuki,Tsunoda,Yuki Tsunoda,https://media.formula1.com/d_driver_fallback_i...,JPN,14,14,10,NaT,NaT,NaT,NaT,+1 Lap,0,2024,United States Grand Prix
2,27,N HULKENBERG,HUL,hulkenberg,Haas F1 Team,B6BABD,haas,Nico,Hulkenberg,Nico Hulkenberg,https://media.formula1.com/d_driver_fallback_i...,GER,8,8,11,NaT,NaT,NaT,0 days 00:01:02.957000,Finished,4,2024,United States Grand Prix
3,20,K MAGNUSSEN,MAG,kevin_magnussen,Haas F1 Team,B6BABD,haas,Kevin,Magnussen,Kevin Magnussen,https://media.formula1.com/d_driver_fallback_i...,DEN,11,11,8,NaT,NaT,NaT,0 days 00:01:19.782000,Finished,0,2024,United States Grand Prix
4,10,P GASLY,GAS,gasly,Alpine,0093cc,alpine,Pierre,Gasly,Pierre Gasly,https://media.formula1.com/d_driver_fallback_i...,FRA,12,12,6,NaT,NaT,NaT,0 days 00:01:30.558000,Finished,0,2024,United States Grand Prix


In [6]:
year=2024

In [7]:
grouped = GP.groupby(['Year','Abbreviation','Position'])["Points"].agg('count').reset_index()
grouped.columns=['Year','Abbreviation','Position','Count']
grouped=grouped[grouped['Year']==year]
grouped=grouped.sort_values(by=['Position','Count'], ascending=[True,False])
grouped.head()

,Year,Abbreviation,Position,Count
7283,2024,VER,1,7
7171,2024,LEC,1,3
7192,2024,NOR,1,3
7149,2024,HAM,1,2
7220,2024,PIA,1,2


In [8]:
fig=px.bar(grouped, 
           x="Position", 
           y='Count', 
           color='Abbreviation',
        #    title="Race Position Counts By Driver for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BEA": "#B6BABD",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LAW": "#6692FF",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            },
            )

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
        margin=dict(l=5, r=5, t=35, b=5)
)
# fig.update_traces(marker={'size': 12})

In [9]:
fig.write_html("/Users/rdesh723/statpulse-html/plots/Race Count by Pos No./{}/Race Final Position Counts By Driver.html".format(year),full_html=False, include_plotlyjs='cdn')

In [10]:
#By Team

In [11]:
grouped_team = GP.groupby(['Year','TeamName','Position'])["Points"].agg('count').reset_index()
grouped_team.columns=['Year','Constructor','Position','Count']
grouped_team=grouped_team[grouped_team['Year']==year]
grouped_team=grouped_team.sort_values(by=['Position','Count'], ascending=[True,False])
grouped_team.head()

,Year,Constructor,Position,Count
12805,2024,Red Bull Racing,1,7
12734,2024,Ferrari,1,5
12770,2024,McLaren,1,5
12780,2024,Mercedes,1,3
12771,2024,McLaren,2,10


In [12]:
fig_team=px.bar(grouped_team, 
           x="Position", 
           y='Count', 
           color='Constructor',
        #    title="Race Position Counts By Team for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            },
            )

fig_team.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    margin=dict(l=5, r=5, t=35, b=5),
)

# fig.update_traces(marker={'size': 12})

In [13]:
fig_team.write_html("/Users/rdesh723/statpulse-html/plots/Race Count by Pos No./{}/Race Final Position Counts By Team.html".format(year),full_html=False, include_plotlyjs='cdn')

In [14]:
#Qualy

In [15]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_qualifying`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2_qualy = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [16]:
GP_qualy=df_bq2_qualy
GP_qualy=GP_qualy.fillna(0)
GP_qualy["Position"]=GP_qualy["Position"].astype(int)
GP_qualy["GridPosition"]=GP_qualy["GridPosition"].astype(int)
GP_qualy["Points"]=GP_qualy["Points"].astype(int)
GP_qualy=GP_qualy[GP_qualy["Position"]!=0]
GP_qualy.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,10,P GASLY,GAS,gasly,Alpine,0093cc,alpine,Pierre,Gasly,Pierre Gasly,https://media.formula1.com/d_driver_fallback_i...,FRA,8,,0,0 days 00:01:17.149000,0 days 00:01:17.048000,0 days 00:01:16.892000,NaT,,0,2024,Mexico City Grand Prix
1,31,E OCON,OCO,ocon,Alpine,0093cc,alpine,Esteban,Ocon,Esteban Ocon,https://media.formula1.com/d_driver_fallback_i...,FRA,19,,0,0 days 00:01:17.617000,NaT,NaT,NaT,,0,2024,Mexico City Grand Prix
2,14,F ALONSO,ALO,alonso,Aston Martin,229971,aston_martin,Fernando,Alonso,Fernando Alonso,https://media.formula1.com/d_driver_fallback_i...,ESP,13,,0,0 days 00:01:17.307000,0 days 00:01:17.168000,NaT,NaT,,0,2024,Mexico City Grand Prix
3,18,L STROLL,STR,stroll,Aston Martin,229971,aston_martin,Lance,Stroll,Lance Stroll,https://media.formula1.com/d_driver_fallback_i...,CAN,14,,0,0 days 00:01:17.407000,0 days 00:01:17.294000,NaT,NaT,,0,2024,Mexico City Grand Prix
4,63,G RUSSELL,RUS,russell,Mercedes,27F4D2,mercedes,George,Russell,George Russell,https://media.formula1.com/d_driver_fallback_i...,GBR,5,,0,0 days 00:01:17.194000,0 days 00:01:16.937000,0 days 00:01:16.356000,NaT,,0,2024,Mexico City Grand Prix


In [17]:
grouped_qualy = GP_qualy.groupby(['Year','Abbreviation','Position'])["Points"].agg('count').reset_index()
grouped_qualy.columns=['Year','Abbreviation','Position','Count']
grouped_qualy=grouped_qualy[grouped_qualy['Year']==year]
grouped_qualy=grouped_qualy.sort_values(by=['Position','Count'], ascending=[True,False])
grouped_qualy.head()

,Year,Abbreviation,Position,Count
5059,2024,VER,1,9
4963,2024,NOR,1,6
4945,2024,LEC,1,2
5011,2024,RUS,1,2
5020,2024,SAI,1,1


In [18]:
fig_qualy=px.bar(grouped_qualy, 
           x="Position", 
           y='Count', 
           color='Abbreviation',
        #    title="Race Position Counts By Driver for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BEA": "#B6BABD",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LAW": "#6692FF",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            },
            )

fig_qualy.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
        margin=dict(l=5, r=5, t=35, b=5)
)
# fig.update_traces(marker={'size': 12})

In [19]:
fig_qualy.write_html("/Users/rdesh723/statpulse-html/plots/Race Count by Pos No./{}/Qualifying Final Position Counts By Driver.html".format(year),full_html=False, include_plotlyjs='cdn')

In [20]:
#By Team

In [21]:
grouped_team_qualy = GP_qualy.groupby(['Year','TeamName','Position'])["Points"].agg('count').reset_index()
grouped_team_qualy.columns=['Year','Constructor','Position','Count']
grouped_team_qualy=grouped_team_qualy[grouped_team_qualy['Year']==year]
grouped_team_qualy=grouped_team_qualy.sort_values(by=['Position','Count'], ascending=[True,False])
grouped_team_qualy.head()

,Year,Constructor,Position,Count
3427,2024,Red Bull Racing,1,9
3389,2024,McLaren,1,6
3354,2024,Ferrari,1,3
3400,2024,Mercedes,1,2
3428,2024,Red Bull Racing,2,8


In [22]:
fig_team_qualy=px.bar(grouped_team_qualy, 
           x="Position", 
           y='Count', 
           color='Constructor',
        #    title="Race Position Counts By Team for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo Racing":"#C92D4B",
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            },
            )

fig_team_qualy.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    margin=dict(l=5, r=5, t=35, b=5),
)

# fig.update_traces(marker={'size': 12})

In [23]:
fig_team_qualy.write_html("/Users/rdesh723/statpulse-html/plots/Race Count by Pos No./{}/Qualifying Final Position Counts By Team.html".format(year),full_html=False, include_plotlyjs='cdn')